In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install langchain langchain-community sentence-transformers langchain_elasticsearch huggingface_hub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.

In [3]:
import os
#from dotenv import load_dotenv

#load_dotenv(verbose=True)
ES_CLOUD_ID = 'f950ade65a814f45b5b8e877d9661527:dXMtY2VudHJhbDEuZ2NwLmNsb3VkLmVzLmlvJGJmYzNmZDY4MjdiOTRhNGVhOGQ1NmY5ODA1YTQ0Y2FlJGNhYzk3NmUyZTMyMzQ0YjJhNTU3MTdjZTk4OTM5Mjhl'
ES_USER = '4177922518'
ES_PASSWORD = 'ghdtjrwn1!'
ES_API_KEY = 'RTVnOUFwRUJHY25xVFJJQWh6bVo6c0JYZm1keGRRblNaU25IM0pQdE56QQ=='
index_name = "helloworld6"

In [4]:
import torch

torch.cuda.empty_cache()

In [5]:
# # qlora embedding

# from peft import PeftModel
# from transformers import AutoTokenizer, AutoModelForCausalLM


In [6]:
!pwd

/content


In [7]:
%cd /content/drive/MyDrive/새싹_공유테스트용/

/content/drive/MyDrive/새싹_공유테스트용


In [8]:
from langchain.embeddings import HuggingFaceEmbeddings

def setup_embeddings():
    # Huggingface embedding setup
    print(">> Prep. Huggingface embedding setup")
    #model_name = "jhgan/ko-sroberta-nli"
    model_name = "MLP-KTLim/llama-3-Korean-Bllossom-8B"
    return HuggingFaceEmbeddings(model_name=model_name, cache_folder="./cache", model_kwargs={"device" : "cuda:0"})

hf = setup_embeddings()

>> Prep. Huggingface embedding setup


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/sett

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [9]:
# ElasticSearch vectorstore in langchain style
from langchain_elasticsearch import ElasticsearchStore

db = ElasticsearchStore(
    es_cloud_id=ES_CLOUD_ID,
    es_user=ES_USER,
    es_password=ES_PASSWORD,
    es_api_key=ES_API_KEY,
    index_name=index_name,
    embedding=hf,
    #es_url = 'https://bfc3fd6827b94a4ea8d56f9805a44cae.us-central1.gcp.cloud.es.io:443'
)

In [10]:
index_name

'helloworld6'

### Data chunking

In [11]:
import pandas as pd
from tqdm import tqdm

# 임시로 경로 변경
data = pd.read_csv('./preprocessed.csv')
#data['text'] = ''
data.head()

,source,title,content
0,법무부 외국인 체류관리,외국인 체류관리,text:\n법무부는 합법적인 이주를 장려하고 촉진하며 지원하기 위하여 다양한 정책...
1,법무부 외국인 체류관리,노동시장의 수요를 고려한 외국인유입정책,"text:\n법무부는 고용노동부, 산업통상자원부 등 관계부처와 유기적으로 협력하여 ..."
2,법무부 외국인 체류관리,계절근로자 제도,text:\n농‧어촌의 고질적인 인력부족 문제를 해소하기 위하여 관계부처와 협업하여...
3,법무부 외국인 체류관리,외국인 숙련기능인력 점수제 비자,text:\n주조‧금형‧용접 등 뿌리산업과 중소 제조업 등 심각한 인력난을 겪고 있...
4,법무부 외국인 체류관리,해외 우수인재 유치,text:\n국가경쟁력에 직결되는 우수인재‧전문인력의 유치를 위해 관리자‧전문직군에...


In [12]:
data.shape

(3433, 3)

In [ ]:
# from langchain.text_splitter import RecursiveCharacterTextSplitter

# text_splitter = RecursiveCharacterTextSplitter(
#     chunk_size = 500,
#     chunk_overlap  = 100,
#     length_function = len,
# )

# #texts = text_splitter.split_text(data[0].page_content)
# texts = text_splitter.split_text(data['content'][0])
# len(texts)


In [13]:
from transformers import AutoTokenizer
# LLaMA-3 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained("MLP-KTLim/llama-3-Korean-Bllossom-8B")

tokenizer_config.json:   0%|          | 0.00/51.1k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/444 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [14]:
from langchain.docstore.document import Document
import pandas as pd
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.documents import Document

# RAG style : title + [SEP] + contents
# seperation 지정되어 있지 않음 -> 공백으로 처리
def make_chunk_data(titles: list, contents: list) -> list:
    if tokenizer.sep_token != None:
        print('## seperated by : ', tokenizer.sep_token)
        chunk_data = [title + tokenizer.sep_token + content for title, content in zip(titles, contents)]
    else:
        chunk_data = [title + " " + content for title, content in zip(titles, contents)]

    return chunk_data


# 토크나이저 기준 분할
def data_chunking(titles: list, contents: list, sources: list) -> Document:
    text_splitter = RecursiveCharacterTextSplitter.from_huggingface_tokenizer(
        tokenizer,
        chunk_size=500,
        chunk_overlap=100
    )
    chunk_data = make_chunk_data(titles, contents)
    chunks = text_splitter.create_documents(chunk_data, sources)

    return chunks

In [19]:
# 빈 리스트 초기화
titles = []
contents = []
sources = []

# 데이터프레임 순회
for row in data:
    titles.append(row['title'])
    contents.append(row['content'])
    sources.append({'source': row['source']})

# 결과 확인
#print("Titles:", titles)
#print("Contents:", contents)

In [21]:
len(titles), len(contents), len(sources)

(3433, 3433, 3433)

In [22]:
chunks = data_chunking(titles, contents, sources)

In [23]:
len(chunks)

6078

In [ ]:



# # RecursiveCharacterTextSplitter 초기화
# text_splitter = RecursiveCharacterTextSplitter.from_huggingface_tokenizer(
#     tokenizer,
#     chunk_size=512,
#     chunk_overlap=100
# )
# chunk_data = make_chunk_data(titles,conents)
# chunks = text_splitter.create_douments(shunk_data)


# # 새로운 DataFrame을 저장할 리스트
# new_rows = []

# # 각 행을 순회하면서 처리
# for _, row in data.iterrows():
#     content = row['content']
#     chunks = text_splitter.split_text(content)

#     # 각 청크에 대해 새로운 행 생성
#     for chunk in chunks:
#         new_row = row.copy()  # 기존 행의 모든 데이터 복사
#         new_row['text'] = chunk  # 'text' 열에 청크 추가
#         new_rows.append(new_row)

# # 새로운 DataFrame 생성
# chunk_data = pd.DataFrame(new_rows)

# # 'content' 열 제거 (선택사항)
# chunk_data = chunk_data.drop(columns=['content'])

# # 인덱스 재설정
# chunk_data = chunk_data.reset_index(drop=True)

# # 결과 확인
# print(chunk_data.head())
# print(f"Total rows in new DataFrame: {len(chunk_data)}")

In [ ]:
# data.shape, chunk_data.shape

In [ ]:
# chunk_data.head()

In [ ]:
# batchtext = list(chunk_data['text'])
# len(batchtext)

In [24]:
chunks[0:2]

[Document(metadata={'source': '법무부 외국인 체류관리'}, page_content='외국인 체류관리 text:\n법무부는 합법적인 이주를 장려하고 촉진하며 지원하기 위하여 다양한 정책적 지원방안을 시행하고 있습니다.\n2013년부터 교수/연구원 등 전문인력, 의료관광객, 단체관광객 등 일부 외국인의 경우, 재외공관을 방문하지 않고도 온라인(비자포털, www.visa.go.kr)으로 대한민국 비자를 신청/발급 받아 신속, 편리하게 입국할 수 있도록 전자비자 제도를 시행함으로써 비자신청 민원편의를 도모하고 있습니다.\n또한 유학비자를 발급받고자 하는 학생들의 경우, 우수한 학습프로그램과 유학생 지원체계를 보유하고 있는 학교를 선택하도록 장려함으로써 우수한 외국 학생들이 한국에서 인적 자본을 성장시킬 수 있도록 지속적으로 지원하고 있습니다.'),
 Document(metadata={'source': '법무부 외국인 체류관리'}, page_content='노동시장의 수요를 고려한 외국인유입정책 text:\n법무부는 고용노동부, 산업통상자원부 등 관계부처와 유기적으로 협력하여 외국인근로자 정책을 수립･시행하고 있습니다.\n농어업 부문 등 1차 산업 및 도장･금형･주조 등 기초 2차 산업의 비숙련 부문 등 인력부족을 겪고 있는 영역에서 외국 인력이 활용될 수 있도록 다양한 비자정책을 시행하고 있습니다.')]

In [25]:
len(chunks)

6078

In [26]:
index_name

'helloworld6'

In [27]:
# """
# 런타임 모두 실행 시 DB에 데이터가 중복 저장되는 것을 막기 위해 주석 처리 해놨습니다.
# 필요한 코드이니 지우지 말아주세요.
# """
#DB에 텍스트 데이터 추가

#꼭 확인!!
index_name = index_name

db.from_documents(chunks,
              embedding=hf,
              es_cloud_id=ES_CLOUD_ID,
              es_user=ES_USER,
              es_password=ES_PASSWORD,
              es_api_key=ES_API_KEY,
              index_name=index_name)